#### 1. Загружаем необходимые библиотеки

In [ ]:
import cv2
from skimage.io import imread, imshow, imsave
from keras.utils.all_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.optimizers import adam_v2
from keras.metrics import Recall
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetB7
import albumentations as A
from albumentations import(
    RandomCrop, ShiftScaleRotate, GaussNoise, MedianBlur,
    HorizontalFlip, VerticalFlip, OneOf, Compose, PadIfNeeded,
    ElasticTransform, RandomSizedBBoxSafeCrop)
import torch
import os
import numpy as np
import pandas as pd
import nums_from_string

#### 2. Подготавливаем исходные данные для дальнейшей детекции

In [ ]:
# Сохраняем отдельно те изображения тренировочного датасета, для которых есть разметка (где есть люди).
# Разметку сохраняем в YOLO формат для последующей детекции в YOLO v5
img_dir = 'C:/ML/Forest/train_dataset_train/train'
labels_out_dir = 'C:/ML/Forest/Augmentations/ini/labels'
images_out_dir = 'C:/ML/Forest/Augmentations/ini/images'

data_df = pd.read_csv("C:/ML/Forest/train_dataset_train/train.csv")
for i in range(len(data_df)):
    line = data_df.iloc[i]
    if line[1] == 0:
        continue
    else:
        img = imread(img_dir + '/' + str(line[0]))
        name = str(line[0]).split('.')[0]
        s = img.shape
        coords = np.array(nums_from_string.get_nums(line[2]))
        coords = coords.reshape((int(coords.shape[0]/3),3))
        imsave(images_out_dir + '/' + line[0], img)
        f = open(labels_out_dir + '/' + name + '.txt', 'w')
        for j in range(int(line[1])):
            coord = coords[j]
            x = coord[0]
            y = coord[1]
            w = 2 * coord[2]
            h = 2 * coord[2]
            out_line = '0 ' + str(x/s[1]) + ' ' + str(y/s[0]) + ' ' + str(w/s[1]) + ' ' + str(h/s[0])
            f.write(out_line + '\n')
        f.close()

In [ ]:
# Аугментируем и сохраняем аугментированные изображения и файлы разметки для дальнейшей детекции в YOLO v5
im_dir = 'C:/ML/Forest/Augmentations/ini/images'
lbl_dir = 'C:/ML/Forest/Augmentations/ini/labels'

im_out = 'C:/ML/Forest/Augmentations/dataset/images'
lbl_out = 'C:/ML/Forest/Augmentations/dataset/labels'

aug_num = 28

for path, directories, files in os.walk(im_dir):
    for file in files:
        filename = file.split(".")[0]
        filepath = path + '/' + file
        img = imread(filepath)
        f = open(lbl_dir + '/' + filename + '.txt', 'r')
        bboxes = []
        for line in f:
            coords = nums_from_string.get_nums(line[1:-1])
            bboxes.append([coords[0], coords[1], coords[2], coords[3], '0'])
        f.close()
        transform = Compose([
                            OneOf([
                                HorizontalFlip(p=1),
                                VerticalFlip(p=1),
                            ], p=0.8),
                            OneOf([
                                ShiftScaleRotate(shift_limit=[-0.2, 0.2], rotate_limit=0, scale_limit=0, p=1, border_mode=cv2.BORDER_REPLICATE),
                                ShiftScaleRotate(shift_limit=0, rotate_limit=[-30, 30], scale_limit=0, p=1, border_mode=cv2.BORDER_REPLICATE),
                                ShiftScaleRotate(shift_limit=0, rotate_limit=0, scale_limit=[0, 0.3], p=1, border_mode=cv2.BORDER_REPLICATE),
                            ], p=1),
                            ], bbox_params=A.BboxParams(format='yolo', min_area=5000), p=1)
        for i in range(aug_num):
            try:
                transformed = transform(image=img, bboxes=bboxes)
                transformed_image = transformed['image']
                transformed_bboxes = transformed['bboxes']
                if transformed_bboxes != []:
                    imsave(im_out + '/' + filename + '_' + str(i + 1) + '.png', transformed_image)
                    f = open(lbl_out + '/' + filename + '_' + str(i + 1) + '.txt', 'w')
                    for j in transformed_bboxes:
                        out_line = '0 %f %f %f %f' % (j[0], j[1], j[2], j[3])
                        f.write(out_line + '\n')
                    f.close()
            except:
                print('wrong bbox')
#        break
#    break

#### 3. Перед детекцией отфильтруем изображения тестового датасета по наличию на них людей с помощью бинарной классификации. Для этого напишем и обучим модель классификации

In [ ]:
# Загружаем изображения тренировочного датасета и их разметки
img_dir = 'C:/ML/Forest/train_dataset_train/train'
size = 512
num_classes = 2

data_df = pd.read_csv("C:/ML/Forest/train_dataset_train/train.csv")
detected_csv = pd.DataFrame(columns=['ID_img', 'count_region', 'region_shape'])
X = []
Y = []
for_weight = []
for i in range(len(data_df)):
    line = data_df.iloc[i]
    img = imread(img_dir + '/' + str(line[0]))
    img = cv2.resize(img, dsize = [size, size], interpolation=cv2.INTER_NEAREST)
    X.append(img)
    lb = line[1]
    if lb > 0:
        lb = 1
    for_weight.append(lb)
    cat_lb = to_categorical(lb, num_classes=num_classes+1, dtype=int)
    cat_lb = np.delete(cat_lb, 2)
    Y.append(cat_lb)
#    break
X = np.array(X)
Y = np.array(Y)
for_weight = np.array(for_weight).astype(int)

In [ ]:
# Рассчитаем весовые коэффициенты обоих классов для балансировки
weight_array = compute_class_weight(class_weight='balanced', classes=np.unique(for_weight), y=for_weight)

weight_dict = dict(zip(np.unique(for_weight), weight_array))
weight_dict = dict(sorted(weight_dict.items()))
new_dict = {}
for i in range(len(weight_dict)):
    new_dict[i] = weight_dict[i]

In [ ]:
# Посмотрим рассчитанные весовые коэффициенты
new_dict

In [ ]:
# Проверим размерность тензора с изображениями
X.shape

In [ ]:
# Проверим размерность тензора с разметкой
Y.shape

In [ ]:
# Разделим тренировочный датасет на тренировочную и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
# Удалим уже неиспользуемые переменные чтобы отчистить оперативную память
del X
del Y

In [ ]:
# Используем предобученную модель классификации
base_model = EfficientNetB7(weights='imagenet', input_shape=(size,size,3), include_top=False)
base_out = base_model.output
base_model.trainable = False

In [ ]:
# Проверим архитектуру предобученной модели
base_model.summary()

In [ ]:
# Допишем голову классификатора, которая будет обучаться на наших выборках
x = base_out
x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
outputs = Dense(num_classes, activation = 'softmax')(x)

In [ ]:
# Собираем модель и проверяем архитектуру
model = Model(inputs=base_model.inputs, outputs=outputs)
model.summary()

In [ ]:
# Компилируем и обучаем модель
model.compile(optimizer=adam_v2.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=["accuracy", Recall()])
model.fit(X_train, Y_train, epochs=5, validation_data=(X_test, Y_test), class_weight=new_dict, batch_size=10, verbose=1)

In [ ]:
# Предсказываем наличие/отсутствие людей на изображениях тестового датасета и сохраняем только те, где есть люди
# для дальнейшей детекции
im_dir = 'C:/ML/Forest/test_dataset_test/test'
detected_out_path = 'C:/ML/Forest/detected'

for path, directories, files in os.walk(im_dir):
    for file in files:
        filename = file.split(".")[0]
        filepath = path + '/' + file
        image = imread(filepath)
        img = cv2.resize(image, dsize = [size, size], interpolation=cv2.INTER_NEAREST)
        test_img_input = np.expand_dims(img, 0)
        prediction = np.squeeze(model.predict(test_img_input))
        prediction = np.argmax(prediction)
        if prediction == 1:
            imsave(detected_out_path + '/' + file, image)
#        break

#### 4. Обучим модель детекции YOLO v5 на ранее аугментированном тренировочном датасете

In [ ]:
# Переходим в рабочую папку модели YOLO v5
%cd C:/ML/Object_detection/Yolo_5/yolov5

In [ ]:
# Обучаем предобученную модель YOLO v5
!python train.py --img 1280 --batch 2 --epochs 50 --data forest_aug.yaml --weights yolov5m.pt --cache

#### 5. Предскажем результат на ранее отфильтрованной классификатором тестовой выборке

In [ ]:
# Загружаем результаты обучения на лучшей эпохе
model = torch.hub.load('C:/ML/Object_detection/Yolo_5/yolov5', 'custom', 'C:/ML/Object_detection/Yolo_5/yolov5/runs/train/exp8/weights/best.pt', source='local')

In [ ]:
# Предсказываем результаты детекции только для тех изображений теста, в которых были найдены люди
# Сохраняем полученные результаты в csv формат, в соответствии с требованиями задачи
test_img_dir = 'C:/ML/Forest/test_dataset_test/test'
df = pd.DataFrame(columns=['ID_img', 'region_shape'])
out_path = 'C:/ML/Forest/Results/Attempt_6/'

for path, directories, files in os.walk('C:/ML/Forest/detected'):
    break

data_df_test = pd.read_csv("C:/ML/Forest/omsk/sample_solution.csv")
for i in range(len(data_df_test)):
    line = data_df_test.iloc[i]
    img = imread(test_img_dir + '/' + line[0])
    coord = '['
    results = model(img)
    res = results.xywh
    res_r = res[0].detach().numpy()
    res = np.around(res_r)
    res = res.astype('int')
    for_sort = []
    for i in range(res.shape[0]):
        x = res[i][0]
        y = res[i][1]
        if res[i][2] <= res[i][3]:
            r = int(round(res[i][2] / 2))
        else:
            r = int(round(res[i][3] / 2))
        for_sort.append([x, y, r])
    for_sort = sorted(for_sort)
    for indx in range(res.shape[0]):
        x = for_sort[indx][0]
        y = for_sort[indx][1]
        r = for_sort[indx][2]
        coord = coord + '\'{"cx":%d,"cy":%d,"r":%d}\'' % (x, y, r) + ','
    l = len(coord)
    coord = coord[:l-1] + ']'
    if coord == ']':
        coord = 0
    if line[0] not in files:
        coord = 0
    new_line = {'ID_img'      : line[0],
                'region_shape': coord}
    df = df.append(new_line, ignore_index=True)
#    break
df.to_csv(out_path + 'Forest_6.csv', index=False)